In [1]:
import gurobipy as gp
import numpy as np
from gurobipy import GRB

from orc.utils import generate_problem
from orc.primal import *
from orc.lagrangean import subgrad_opt

In [2]:
A, b = generate_problem(20, 50)

In [3]:
X0 = np.array([False] * A.shape[-1])
X1 = np.array([False] * A.shape[-1])

In [4]:
x = hall_hochbaum(A, b)
ub = np.sum(A, axis=0) @ x
ub, x

(35201.0,
 array([0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 1.,
        1., 1., 1., 1., 0., 0., 0., 1., 0., 0., 1., 1., 1., 1., 1., 1., 0.,
        0., 1., 1., 0., 0., 1., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0.]))

In [ ]:
lambd, lb, x_lb = subgrad_opt(A, b, ub, X0, X1)
lambd, lb, x_lb

In [ ]:
list(zip(A @ x_lb - b, lambd))

In [ ]:
list(zip((1 - lambd) @ A, x_lb))

In [ ]:
np.sum(A, axis=0)

In [ ]:
m = gp.Model("mip")
x = m.addMVar(A.shape[-1], vtype=GRB.BINARY, name="x")
m.setObjective(np.sum(A, axis=0) @ x)
m.addConstr(A @ x >= b)

In [ ]:
m.optimize()
for v in m.getVars():
    print(v.varName, v.x)
m.getObjective().getValue()

In [ ]:
r = np.argmax(b - A @ x_lb)
r

In [ ]:
fixed0 = set(np.where(X0)[0])
fixed1 = set(np.where(X1)[0])
zero_entry = set(np.where(A[r] == 0)[0])
not_candidates = list(fixed0.union(fixed1).union(zero_entry))
not_candidates

In [ ]:
cr = (1 - lambd) @ A
cr

In [ ]:
cr[not_candidates] = np.inf
cr

In [ ]:
j = np.argmin(cr)
j
# add j to X0 in a branch and to X1 in the other